In [1]:
%%writefile psum.py
from openmpi.mpi4py import MPI
import numpy as np

def psum(a):
    locsum = np.sum(a)
    rcvBuf = np.array(0.0,'d')
    MPI.COMM_WORLD.Allreduce([locsum, MPI.DOUBLE],
        [rcvBuf, MPI.DOUBLE],
        op=MPI.SUM)
    return rcvBuf



Overwriting psum.py


In [9]:
from ipyparallel import Client
from openmpi.mpi4py import MPI
import numpy as np

In [10]:
c = Client(profile='default')

In [11]:
view = c[:]
print (view)

<DirectView [0, 1, 2, 3]>


In [12]:
view.activate()

In [13]:
view.run('psum.py')

<AsyncResult: execute>

In [14]:
view.scatter('a',np.arange(200,dtype='float'))

<AsyncResult: scatter>

In [15]:
view['a']

[array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.]),
 array([ 50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
         61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
         72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,
         83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,
         94.,  95.,  96.,  97.,  98.,  99.]),
 array([ 100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,  108.,
         109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,  117.,
         118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,  126.,
         127.,  128.,  129.,  130.,  131.,  132.,  133.,  134.,  135.,
         136.,  137.,  138.,  13

In [16]:
%px totalsum = psum(a)

<AsyncResult: execute>

In [17]:
r = view['totalsum']

In [18]:
print(r[0])

1225.0


In [19]:
print(r)

[array(1225.0), array(3725.0), array(6225.0), array(8725.0)]


In [7]:
%%writefile hello.py
from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
threads = comm.Get_size()
data = np.zeros(1)
if rank == 0:
    for i in range(threads):
        data[0] = i*100
        MPI.COMM_WORLD.Send(data, dest = i)
else:
    print(data[0])
    MPI.COMM_WORLD.Recv(data, source = 0)
    print(data[0])
    data[0] = data[0] + 4
    print("hello world from process ", rank, threads)
    
    
    MPI.COMM_WORLD.Send(data, dest = 0)
    
if rank == 0:
    for i in range(threads):
        MPI.COMM_WORLD.Recv(data, source = ANY_SOURCE)
        print(data[0])
        

Overwriting hello.py


In [8]:
!mpiexec  -n 4 python hello.py

0.0
100.0
0.0
('hello world from process ', 1, 4)
104.0
0.0
200.0
('hello world from process ', 2, 4)
204.0
0.0
300.0
('hello world from process ', 3, 4)
304.0


In [18]:
import numpy as np
from ipyparallel import Client

c = Client(profile='default')
dview = c[:]

A = np.random.random(64)

@dview.parallel(block=True)
def pmul(A,B):
    return (A*B)

C_local = A*A
C_remote = pmul(A,A)
(C_local == C_remote).all()

True

In [17]:
print (C_remote)

[  4.88651963e-03   1.52182823e-01   4.48463589e-01   4.54994437e-02
   2.59274086e-01   6.17351127e-02   7.01398386e-03   4.71226081e-01
   7.13436721e-01   1.02745301e-01   1.73863392e-02   2.45134090e-01
   7.49513788e-03   1.24371576e-01   9.20788916e-01   6.75025199e-01
   3.96993334e-01   2.22785334e-01   4.40018565e-01   3.12086402e-02
   1.33680054e-01   1.43382277e-03   2.14089253e-01   9.93123606e-01
   9.20783406e-01   3.56604431e-02   1.21113103e-01   1.69397882e-01
   5.87813486e-01   6.44559009e-01   1.50275858e-02   6.62652037e-03
   1.40887367e-01   8.61745293e-01   3.04769857e-01   7.75474250e-01
   2.10810076e-02   9.39368933e-01   3.32233911e-01   3.35828527e-01
   2.46173052e-01   8.26162938e-01   1.75610076e-02   2.47386974e-02
   1.67524211e-01   2.39128363e-01   9.91380460e-01   3.57577504e-01
   1.69163270e-01   9.76310577e-02   4.99768103e-01   5.43170403e-02
   4.52298390e-01   1.77651903e-01   2.30299017e-01   7.83056593e-01
   4.63485024e-04   8.31674146e-02

In [22]:
@dview.parallel(block=True)
def echo(x):
    return x

echo(range(8))

[range(0, 2), range(2, 4), range(4, 6), range(6, 8)]

In [24]:
echo.map(range(8))

[0, 1, 2, 3, 4, 5, 6, 7]

In [25]:
a = range(100)
print(a[0:10])

range(0, 10)
